# Credit Card Applications
Commercial banks receive a lot of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this notebook, we will build an automatic credit card approval predictor using machine learning techniques, just like the real banks do!

We'll use the Credit Card Approval dataset from the UCI Machine Learning Repository. The structure of this notebook is as follows:

 - First, we will start off by loading and viewing the dataset.
 - We will see that the dataset has a mixture of both numerical and non- numerical features, that it contains values from different ranges, plus that it contains a number of missing entries.
 - We will have to preprocess the dataset to ensure the machine learning model we choose can make good predictions.
 - After our data is in good shape, we will do some exploratory data analysis to build our intuitions.
 - Finally, we will build a machine learning model that can predict if an individual's application for a credit card will be accepted.
 

First, loading and viewing the dataset. We find that since this data is confidential, the contributor of the dataset has anonymized the feature names.

In [1]:
import pandas as pd

In [4]:
cc_apps = pd.read_csv("C:/Users/UDAYAN/Desktop/Projects/Python/Credit Cards Approval/cc_approvals.csv", header = None)
cc_apps.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.250,t,t,1,f,g,202,0,+
1,a,58.67,4.460,u,g,q,h,3.040,t,t,6,f,g,43,560,+
2,a,24.5,0.500,u,g,q,h,1.500,t,f,0,f,g,280,824,+
3,b,27.83,1.540,u,g,w,v,3.750,t,t,5,t,g,100,3,+
4,b,20.17,5.625,u,g,w,v,1.710,t,f,0,f,s,120,0,+
5,b,32.08,4.000,u,g,m,v,2.500,t,f,0,t,g,360,0,+
6,b,33.17,1.040,u,g,r,h,6.500,t,f,0,t,g,164,31285,+
7,a,22.92,11.585,u,g,cc,v,0.040,t,f,0,f,g,80,1349,+
8,b,54.42,0.500,y,p,k,h,3.960,t,f,0,f,g,180,314,+
9,b,42.5,4.915,y,p,w,v,3.165,t,f,0,t,g,52,1442,+


# Inspecting the Applications
The output may appear a bit confusing at its first sight, but let's try to figure out the most important features of a credit card application. The features of this dataset have been anonymized to protect the privacy, but this blog gives us a pretty good overview of the probable features. The probable features in a typical credit card application are Gender, Age, Debt, Married, BankCustomer, EducationLevel, Ethnicity, YearsEmployed, PriorDefault, Employed, CreditScore, DriversLicense, Citizen, ZipCode, Income and finally the ApprovalStatus. This gives us a pretty good starting point, and we can map these features with respect to the columns in the output.

As we can see from our first glance at the data, the dataset has a mixture of numerical and non-numerical features. This can be fixed with some preprocessing, but before we do that, let's learn a bit more about the dataset a bit more to see if there are other dataset issues that need to be fixed.

In [8]:
cc_apps_description = cc_apps.describe()
print(cc_apps_description)
print("\n")

cc_apps_info = cc_apps.info()
print(cc_apps_info)
print("\n")

cc_apps.tail(20)

               2           7          10             14
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
0     690 non-null object
1     690 non-null object
2     690 non-null float64
3     690 non-null object
4     690 non-null object
5     690 non-null object
6     690 non-null object
7     690 non-null float64
8     690 non-null object
9     690 non-null object
10    690 non-null int64
11    690 non-null object
12    690 non-null object
13    690 non-null object
14    690 non-null int64

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
670,b,47.17,5.835,u,g,w,v,5.500,f,f,0,f,g,465,150,-
671,b,25.83,12.835,u,g,cc,v,0.500,f,f,0,f,g,0,2,-
672,a,50.25,0.835,u,g,aa,v,0.500,f,f,0,t,g,240,117,-
673,?,29.5,2.000,y,p,e,h,2.000,f,f,0,f,g,256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,0,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,0,0,-


# Handling the missing values (Part 1)
We've uncovered some issues that will affect the performance of our machine learning model(s) if they go unchanged:

 - Our dataset contains both numeric and non-numeric data (specifically data that are of float64, int64 and object types). Specifically, the features 2, 7, 10 and 14 contain numeric values (of types float64, float64, int64 and int64 respectively) and all the other features contain non-numeric values.
 - The dataset also contains values from several ranges. Some features have a value range of 0 - 28, some have a range of 2 - 67, and some have a range of 1017 - 100000. Apart from these, we can get useful statistical information (like mean, max, and min) about the features that have numerical values.
 - Finally, the dataset has missing values, which we'll take care of in this task. The missing values in the dataset are labeled with '?', which can be seen in the last cell's output.
Now, let's temporarily replace these missing value question marks with NaN.

In [9]:
import numpy as np

In [12]:
print(cc_apps.tail(20))
cc_apps = cc_apps.replace('?', np.NaN)
print(cc_apps.tail(20))

      0      1       2  3  4   5   6      7  8  9   10 11 12   13   14 15
670    b  47.17   5.835  u  g   w   v  5.500  f  f   0  f  g  465  150  -
671    b  25.83  12.835  u  g  cc   v  0.500  f  f   0  f  g    0    2  -
672    a  50.25   0.835  u  g  aa   v  0.500  f  f   0  t  g  240  117  -
673  NaN   29.5   2.000  y  p   e   h  2.000  f  f   0  f  g  256   17  -
674    a  37.33   2.500  u  g   i   h  0.210  f  f   0  f  g  260  246  -
675    a  41.58   1.040  u  g  aa   v  0.665  f  f   0  f  g  240  237  -
676    a  30.58  10.665  u  g   q   h  0.085  f  t  12  t  g  129    3  -
677    b  19.42   7.250  u  g   m   v  0.040  f  t   1  f  g  100    1  -
678    a  17.92  10.210  u  g  ff  ff  0.000  f  f   0  f  g    0   50  -
679    a  20.08   1.250  u  g   c   v  0.000  f  f   0  f  g    0    0  -
680    b   19.5   0.290  u  g   k   v  0.290  f  f   0  f  g  280  364  -
681    b  27.83   1.000  y  p   d   h  3.000  f  f   0  f  g  176  537  -
682    b  17.08   3.290  u  g   i   v 

# Handling the missing values (Part 2)
We have successfully taken care of the missing values present in the numeric columns. There are still some missing values to be imputed for columns 0, 1, 3, 4, 5, 6 and 13. All of these columns contain non-numeric data and this why the mean imputation strategy would not work here. This needs a different treatment.

We are going to impute these missing values with the most frequent values as present in the respective columns. This is good practice when it comes to imputing missing values for categorical data in general.

In [17]:
cc_apps.fillna(cc_apps.mean(), inplace = True)

In [18]:
cc_apps.isnull()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
686,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
687,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
688,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [19]:
cc_apps.isnull().sum()

0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13    13
14     0
15     0
dtype: int64

In [21]:
cc_apps.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], dtype='int64')

# Handling the missing values (Part 3)
We have successfully taken care of the missing values present in the numeric columns. There are still some missing values to be imputed for columns 0, 1, 3, 4, 5, 6 and 13. All of these columns contain non-numeric data and this why the mean imputation strategy would not work here. This needs a different treatment.

We are going to impute these missing values with the most frequent values as present in the respective columns. This is good practice when it comes to imputing missing values for categorical data in general.

In [23]:
for col in cc_apps.columns:
    if cc_apps[col].dtypes == 'object':
        cc_apps[col] = cc_apps[col].fillna(cc_apps[col].value_counts().index[0])

        
cc_apps.isnull().sum()        

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
dtype: int64

# Preprocessing the data (Part 1)
The missing values are now successfully handled.

There is still some minor but essential data preprocessing needed before we proceed towards building our machine learning model. We are going to divide these remaining preprocessing steps into two main tasks:

 - Convert the non-numeric data into numeric.
 - Scale the feature values to a uniform range.
 
First, we will be converting all the non-numeric values into numeric ones. We do this because not only it results in a faster computation but also many machine learning models (like XGBoost) (and especially the ones developed using scikit-learn) require the data to be in a strictly numeric format. We will do this by using a technique called label encoding.

In [26]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#le.fit(["tokyo", "paris"])
#le.transform(["tokyo", "tokyo", "paris"])

In [28]:
for col in cc_apps.columns:
    if cc_apps[col].dtype == 'object':
        #le.fit_transform(["tokyo", "tokyo", "paris"])
        cc_apps[col] = le.fit_transform(cc_apps[col])

# Preprocessing the data (Part 2)
We have successfully converted all the non-numeric values to numeric ones.

Now, let's try to understand what these scaled values mean in the real world. Let's use CreditScore as an example. The creidt score of a person is their creditworthiness based on their credit history. The higher this number, the more financially trustworthy a person is considered to be. So, a CreditScore of 1 is the highest since we're rescaling all the values to the range of 0-1.

Also, features like DriversLicense and ZipCode are not as important as the other features in the dataset for predicting credit card approvals. We should drop them to design our machine learning model with the best set of features. This is often called feature engineering or, more specifically, feature selection.

In [29]:
from sklearn.preprocessing import MinMaxScaler

In [30]:
cc_apps = cc_apps.drop([10, 13], axis = 1)
cc_apps = cc_apps.values

In [31]:
x, y = cc_apps[:, 0:13] , cc_apps[:, 13]

In [34]:
#print(x)
#print(y)

In [40]:
scaler = MinMaxScaler(feature_range=(0,1))
rescaledx = scaler.fit_transform(x)

# Splitting the dataset into Training and Test Sets
Now that we have our data in a machine learning modeling-friendly shape, we are really ready to proceed towards creating a machine learning model to predict which credit card applications will be accepted and which will be rejected.

First, we will split our data into train set and test set to prepare our data for two different phases of machine learning modeling: training and testing.

In [41]:
from sklearn.model_selection import train_test_split

In [42]:
x_train, x_test, y_train, y_test = train_test_split(rescaledx, y, test_size = .33, random_state = 42)

# Fitting a Logistic Regression Model to the Train Set
Essentially, predicting if a credit card application will be approved or not is a classification task. According to UCI, our dataset contains more instances that correspond to "Denied" status than instances corresponding to "Approved" status. Specifically, out of 690 instances, there are 383 (55.5%) applications that got denied and 307 (44.5%) applications that got approved.

This gives us a benchmark. A good machine learning model should be able to accurately predict the status of the applications with respect to these statistics.

Which model should we pick? A question to ask is: are the features that affect the credit card approval decision process correlated with each other? Although we can measure correlation, that is outside the scope of this notebook, so we'll rely on our intuition that they indeed are correlated for now. Because of this correlation, we'll take advantage of the fact that generalized linear models perform well in these cases. Let's start our machine learning modeling with a Logistic Regression model (a generalized linear model).

In [43]:
from sklearn.linear_model import LogisticRegression

In [44]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)

C:\Users\UDAYAN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

# Making Predictions and Evaluating Performance
But how well does our model perform?

We will now evaluate our model on the test set with respect to classification accuracy. But we will also take a look the model's confusion matrix. In the case of predicting credit card applications, it is equally important to see if our machine learning model is able to predict the approval status of the applications as denied that originally got denied. If our model is not performing well in this aspect, then it might end up approving the application that should have been approved. The confusion matrix helps us to view our model's performance from these aspects.

In [45]:
from sklearn.metrics import confusion_matrix

In [46]:
y_pred = logreg.predict(x_test)

In [48]:
print("Accuracy of Logistic Regression Classifier:", logreg.score(x_test, y_test))
confusion_matrix(y_test, y_pred)

Accuracy of Logistic Regression Classifier: 0.8333333333333334


array([[92, 11],
       [27, 98]], dtype=int64)

# Grid searching and making the model perform better
Our model was pretty good! It was able to yield an accuracy score of almost 84%.

For the confusion matrix, the first element of the of the first row of the confusion matrix denotes the true negatives meaning the number of negative instances (denied applications) predicted by the model correctly. And the last element of the second row of the confusion matrix denotes the true positives meaning the number of positive instances (approved applications) predicted by the model correctly.

Let's see if we can do better. We can perform a grid search of the model parameters to improve the model's ability to predict credit card approvals.

scikit-learn's implementation of logistic regression consists of different hyperparameters but we will grid search over the following two:

 - tol
 - max_iter

In [49]:
from sklearn.model_selection import GridSearchCV

In [50]:
tol = [.01, .001, .0001]
max_iter = [100, 150, 200]

In [51]:
param_grid = dict(tol = tol, max_iter = max_iter)

# Finding the best Performing Model
We have defined the grid of hyperparameter values and converted them into a single dictionary format which GridSearchCV() expects as one of its parameters. Now, we will begin the grid search to see which values perform best.

We will instantiate GridSearchCV() with our earlier logreg model with all the data we have. Instead of passing train and test sets, we will supply rescaledX and y. We will also instruct GridSearchCV() to perform a cross-validation of five folds.

We'll end the notebook by storing the best-achieved score and the respective best parameters.

While building this credit card predictor, we tackled some of the most widely-known preprocessing steps such as scaling, label encoding, and missing value imputation. We finished with some machine learning to predict if a person's application for a credit card would get approved or not given some information about that person.

In [53]:
grid_model = GridSearchCV(estimator = logreg, param_grid = param_grid, cv = 5)
grid_model_result = grid_model.fit(rescaledx, y)

C:\Users\UDAYAN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\UDAYAN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\UDAYAN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\UDAYAN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\UDAYAN\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to sile

In [55]:
best_score, best_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best: %f using %s" %(best_score, best_params))

Best: 0.855072 using {'max_iter': 100, 'tol': 0.01}
